In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import glob
%cd ../

/home/krong/Documents/classes/SP/final/AdaptiveAccompaniment


In [2]:
fileId_path = "experiments/fileIds.pkl"
queryInfo_path = "experiments/queryInfo.pkl"
timeData_path = "experiments/timeData.pkl"
audioFiles_path = "experiments/audioFiles.pkl"
with open(fileId_path, 'rb') as f:
    fileIds=pkl.load(f)
with open(queryInfo_path, 'rb') as f:
    queryInfo=pkl.load(f)
with open(timeData_path, 'rb') as f:
    timeData=pkl.load(f)
with open(audioFiles_path, 'rb') as f:
    audioFiles=pkl.load(f)

In [3]:
def readHypFiles(hypdir):
    data = []
    for hypfile in sorted(glob.glob("experiments/{}/*.hyp".format(hypdir))):
        print(hypfile)
        with open(hypfile, "rb") as f:
            data.append(pkl.load(f))
    return data

In [4]:
def evaluateSegmentLevel(segment_predict, solo_name, ref_name, sr=22050):
    pieceName = solo_name.split('_')[0]
    gt_segments = queryInfo[pieceName]
    gt_samples = []
    diffs = []
    for beats in gt_segments:
        beat1 = beats.split(',')[0][1:]
        beat2 = beats.split(',')[1][:-1]
        sample1 = timeData[ref_name][beat1]*512
        sample2 = timeData[ref_name][beat2]*512
        gt_samples.append((sample1, sample2))
    for idx,segment in enumerate(segment_predict):
        seg0 = segment[0]
        seg1 = segment[1]
        diff0 = np.abs(gt_samples[idx][0]-seg0)/sr
        diff1 = np.abs(gt_samples[idx][1]-seg1)/sr
        diffs.append(diff0)
        diffs.append(diff1)
    return diffs

In [56]:
def evaluateBeatLevel(tf, segment_predict, solo_name, ref_name, sr=22050):
    # Only evaluate within segments
    diffs = []
    beat_ls = []
    pieceName = solo_name.split('_')[0]
    gt_segments = queryInfo[pieceName]
    annotated_beats = list(timeData[ref_name].keys())
    for beats in gt_segments:
        beat1 = beats.split(',')[0][1:]
        beat2 = beats.split(',')[1][:-1]
        idx1 = annotated_beats.index(beat1)
        idx2 = annotated_beats.index(beat2)
        base_time = timeData[ref_name][beat1]
        new_times = {}
        for i in range(idx1,idx2):
            key = annotated_beats[i]
            sample = (timeData[ref_name][key]-base_time)*sr
            new_time = tf[int(sample)]
            new_times[key]=new_time
        
        gt_beats = list(timeData[solo_name].keys())
        gt_base_time=timeData[solo_name][beat1]
        for key in new_times:
            if key in gt_beats:
                gt_time = (timeData[solo_name][key]-gt_base_time)*sr
                pred_time = new_times[key]
                #print(gt_time, pred_time)
                diffs.append(np.abs(gt_time-pred_time))
        
        gt_beats = list(timeData[solo_name].keys())
        gt_base_time=timeData[solo_name][beat1]
        for key in new_times:
            if key in gt_beats:
                gt_time = (timeData[solo_name][key]-gt_base_time)*sr
                pred_time = new_times[key]
                #print(gt_time/sr, pred_time/sr)
                diffs.append(np.abs(gt_time-pred_time))
                
    return diffs

In [57]:
def evaluateDiffLevel(tf, segment_predict, solo_name, ref_name, sr=22050):
    # Only evaluate within segments
    diffs = []
    beat_ls = []
    pieceName = solo_name.split('_')[0]
    gt_segments = queryInfo[pieceName]
    annotated_beats = list(timeData[ref_name].keys())
    for beats in gt_segments:
        beat1 = beats.split(',')[0][1:]
        beat2 = beats.split(',')[1][:-1]
        idx1 = annotated_beats.index(beat1)
        idx2 = annotated_beats.index(beat2)
        base_time = timeData[ref_name][beat1]
        new_times = {}
        for i in range(idx1,idx2):
            key = annotated_beats[i]
            sample = (timeData[ref_name][key]-base_time)*sr
            new_time = tf[int(sample)]
            new_times[key]=new_time
        
        gt_beats = list(timeData[solo_name].keys())
        gt_base_time=timeData[solo_name][beat1]
        for key in new_times:
            if key in gt_beats:
                gt_time = (timeData[solo_name][key]-gt_base_time)*sr
                pred_time = new_times[key]
                #print(gt_time/sr, pred_time/sr)
                #diffs.append(np.abs(gt_time-pred_time))
                beat_array.append((gt_time,pred_time))
    
        print('here')
    beat_ls.append(beat_array)
    for i in range(len(beat_ls)):
        for j in range(len(beat_ls[i])):
            if j>0:
                ground_diff = beat_ls[i][j][0]-beat_ls[i][j-1][0]
                predict_diff = beat_ls[i][j][1]-beat_ls[i][j-1][1]
                diffs.append(np.abs(ground_diff-predict_diff))
    
    return diffs

In [58]:
def evaluate(hypdir, sr=22050):
    data = readHypFiles(hypdir)
    times = []
    segment_tolerances =[1]
    segment_total = np.zeros(len(segment_tolerances))
    segment_correct = np.zeros(len(segment_tolerances))
    
    beat_tolerances =[.25]
    beat_total = np.zeros(len(beat_tolerances))
    beat_correct = np.zeros(len(beat_tolerances))
    
    diff_tolerances =[.25]
    diff_total = np.zeros(len(diff_tolerances))
    diff_correct = np.zeros(len(diff_tolerances))
    for solo_name, ref_name, tf, segment_predict, time_taken in data:
        segment_predict = np.array(segment_predict)*512
        segment_diffs = evaluateSegmentLevel(segment_predict, solo_name, ref_name)
        beat_diffs = evaluateBeatLevel(tf, segment_predict, solo_name, ref_name)
        diff_diffs = evaluateDiffLevel(tf, segment_predict, solo_name, ref_name)
        segment_diffs =np.array(segment_diffs)/sr
        beat_diffs =np.array(beat_diffs)/sr
        diff_diffs = np.array(diff_diffs)/sr
        times.append(time_taken)
        
        #Update Segment Level Accuracy
        for idx, tolerance in enumerate(segment_tolerances):
            segment_correct[idx] += np.sum(np.where(segment_diffs<tolerance, 1, 0))
            segment_total[idx]+=len(segment_diffs)
            
            
        #Update Beat Level Accuracy
        for idx, tolerance in enumerate(beat_tolerances):
            beat_correct[idx] += np.sum(np.where(beat_diffs<tolerance, 1, 0))
            beat_total[idx]+=len(beat_diffs)
            
        for idx, tolerance in enumerate(diff_tolerances):
            diff_correct[idx] += np.sum(np.where(diff_diffs<tolerance, 1, 0))
            diff_total[idx]+=len(diff_diffs)
    print(segment_correct/segment_total)
    print(beat_correct/beat_total)
    print(diff_correct/diff_total)